In [3]:
import pandas as pd
from random import randint
import math
from gvgen import *
import sys

In [4]:
def generateGraph(nodes, linking):
    graph = dict()
    for i in range(nodes):
        graph[i] = []
    for i in range(nodes):
        for j in range(nodes):
            if i == j:
                continue
            prob = randint(1,int(linking.as_integer_ratio()[1]))
            if(prob <= int(linking.as_integer_ratio()[0])):
                graph[i].append(j)
    return graph

In [5]:
smallGraph = generateGraph(20,.1)
totalDegree = 0


smallGraph

{0: [],
 1: [8, 16],
 2: [],
 3: [6, 12],
 4: [10, 11],
 5: [10, 18],
 6: [17],
 7: [2, 8, 12],
 8: [0],
 9: [4, 18],
 10: [11, 15],
 11: [],
 12: [6],
 13: [],
 14: [0, 19],
 15: [3],
 16: [],
 17: [1, 3, 16],
 18: [3, 4, 13],
 19: [4, 7, 14, 16]}

In [6]:
graph = GvGen()
gvComp = dict()
even = graph.newItem("Even")
odd = graph.newItem("Odd")
for startN in smallGraph:
    if startN not in gvComp:
        if startN%2 == 0:
            gvComp[startN] = graph.newItem(str(startN), even)
        else:
            gvComp[startN] = graph.newItem(str(startN), odd)
    for endN in smallGraph[startN]:
        if endN %2 == startN %2:
            continue
        if endN not in gvComp:
            if endN %2 == 0:
                gvComp[endN] = graph.newItem(str(endN), even)
            else:
                gvComp[endN] = graph.newItem(str(endN), odd)
        graph.newLink(gvComp[startN],gvComp[endN])





In [7]:
for startN in gvComp:
    if startN % 2 ==0:
        graph.propertyAppend(gvComp[startN], "color", "blue")
        graph.propertyAppend(gvComp[startN], "style", "filled")
        graph.propertyAppend(gvComp[startN], "fontcolor", "white")
    else:
        graph.propertyAppend(gvComp[startN], "color", "red")
        graph.propertyAppend(gvComp[startN], "shape", "rectangle")


graph.dot()

/* Generated by GvGen v.1.0 (https://www.github.com/stricaud/gvgen) */

digraph G {
compound=true;
   subgraph cluster1 {
   label="Even";
      node3 [label="0",color="blue",style="filled",fontcolor="white"];
      node5 [label="8",color="blue",style="filled",fontcolor="white"];
      node6 [label="16",color="blue",style="filled",fontcolor="white"];
      node7 [label="2",color="blue",style="filled",fontcolor="white"];
      node9 [label="6",color="blue",style="filled",fontcolor="white"];
      node10 [label="12",color="blue",style="filled",fontcolor="white"];
      node11 [label="4",color="blue",style="filled",fontcolor="white"];
      node14 [label="10",color="blue",style="filled",fontcolor="white"];
      node15 [label="18",color="blue",style="filled",fontcolor="white"];
      node21 [label="14",color="blue",style="filled",fontcolor="white"];
   }
   subgraph cluster2 {
   label="Odd";
      node4 [label="1",color="red",shape="rectangle"];
      node8 [label="3",color="red",shape="

In [8]:
import random

# Modified generateGraph function to support multigraphs and weights
def generateGraph(nodes, linking, multigraph=False, weighted=False):
    graph = dict()
    for i in range(nodes):
        graph[i] = []
    for i in range(nodes):
        for j in range(nodes):
            if i == j:
                continue
            prob = random.randint(1, int(linking.as_integer_ratio()[1]))
            if prob <= int(linking.as_integer_ratio()[0]):
                if multigraph:
                    # Allow multiple edges by adding the same edge more than once
                    repetitions = random.randint(1, 3)  # Up to 3 edges for demonstration
                    for _ in range(repetitions):
                        if weighted:
                            weight = random.randint(1, 10)
                            graph[i].append((j, weight))
                        else:
                            graph[i].append(j)
                else:
                    if weighted:
                        weight = random.randint(1, 10)
                        graph[i].append((j, weight))
                    else:
                        graph[i].append(j)
    return graph

# Generate graph data
multigraphData = generateGraph(5, 0.5, multigraph=True)  # Smaller graph for clarity
weightedGraphData = generateGraph(5, 0.5, weighted=True)

# Function to create a DOT file from the graph data
def createDotFile(graphData, fileName, multigraph=False, weighted=False):
    with open(fileName, 'w') as f:
        f.write("digraph G {\n")
        for node, edges in graphData.items():
            for edge in edges:
                if weighted:
                    targetNode, weight = edge
                    f.write(f'    "{node}" -> "{targetNode}" [label="{weight}"];\n')
                else:
                    f.write(f'    "{node}" -> "{edge}";\n')
        f.write("}\n")

# Create DOT files
createDotFile(multigraphData, "multigraph.dot", multigraph=True)
createDotFile(weightedGraphData, "weightedGraph.dot", weighted=True)


In [9]:
def calculate_percent_difference(node1, node2):
    # Simple metric for demonstration
    percent_difference = abs(node1 - node2) / float(max(node1, node2) + 1) * 100  # Avoid division by zero
    return round(percent_difference, 2)

# Define bucket ranges (0-25%, 25-50%, 50-75%, 75-100%)
buckets = [(0, 25), (25, 50), (50, 75), (75, 100)]

def create_bucketed_dot(graphData, fileName):
    with open(fileName, 'w') as f:
        f.write("digraph G {\n")
        f.write('    rankdir=LR;\n')  # Lay out the graph from left to right
        f.write('    node [shape=box];\n')  # Use box shape for nodes
        
        # Create a subgraph for each bucket
        for i, (min_percent, max_percent) in enumerate(buckets, start=1):
            f.write(f'    subgraph cluster_{i} {{\n')
            f.write(f'        label="Bucket {i}: {min_percent}% to {max_percent}%";\n')
            f.write(f'        color=blue;\n')
            
            # Add edges that fall into the current bucket
            for node, edges in graphData.items():
                for target in edges:
                    percent_diff = calculate_percent_difference(node, target)
                    if min_percent <= percent_diff <= max_percent:
                        # Use percent_diff as weight
                        f.write(f'        "{node}" -> "{target}" [label="{percent_diff}%", weight="{percent_diff}"];\n')
            
            f.write('    }\n')  # Close subgraph
        
        f.write("}\n")  # Close main graph

create_bucketed_dot(smallGraph, "bucketedGraph.dot")


In [12]:
def calculate_percent_difference(node1, node2):
    # Simple metric for demonstration
    percent_difference = abs(node1 - node2) / float(max(node1, node2) + 1) * 100  # Avoid division by zero
    return round(percent_difference, 2)

# Define bucket ranges (0-25%, 25-50%, 50-75%, 75-100%)
buckets = [(0, 25), (25, 50), (50, 75), (75, 100)]
threshold = 15  # Threshold for highlighting edges

def create_bucketed_dot_with_threshold(graphData, fileName):
    with open(fileName, 'w') as f:
        f.write("digraph G {\n")
        f.write('    rankdir=LR;\n')  # Lay out the graph from left to right
        f.write('    node [shape=box];\n')  # Use box shape for nodes
        
        # Create a subgraph for each bucket
        for i, (min_percent, max_percent) in enumerate(buckets, start=1):
            f.write(f'    subgraph cluster_{i} {{\n')
            f.write(f'        label="Bucket {i}: {min_percent}% to {max_percent}%";\n')
            f.write(f'        color=blue;\n')
            
            # Add edges that fall into the current bucket
            for node, edges in graphData.items():
                for target in edges:
                    percent_diff = calculate_percent_difference(node, target)
                    if min_percent <= percent_diff <= max_percent:
                        edge_style = 'color=red' if percent_diff < threshold else ''  # Mark edge in red if above threshold
                        f.write(f'        "{node}" -> "{target}" [label="{percent_diff}%", weight="{percent_diff}" {edge_style}];\n')
            
            f.write('    }\n')  # Close subgraph
        
        f.write("}\n")  # Close main graph

# Assuming smallGraph is already defined as per your dataset
create_bucketed_dot_with_threshold(smallGraph, "thresholdGraph.dot")
